In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline

from acquire import get_titanic_data
from prepare import prep_titanic_data

df = get_titanic_data()
df = prep_titanic_data(df)

# Handle missing values in the `age` column.
df.dropna(inplace=True)

In [2]:
# for saga solver:
X_train = [] # this line is so I can rerun this code without error
df = get_titanic_data()
df = prep_titanic_data(df)
# Handle missing values in the `age` column.
df.dropna(inplace=True)
X = df[['pclass','age','fare','sibsp','parch']]
y = df[['survived']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

# 1. make the thing
scaler = MinMaxScaler()

# 2. fit the thing
scaler.fit(X_train[['age', 'fare']])

# 3. use the thing
X_train[['age', 'fare']] = scaler.transform(X_train[['age', 'fare']])
X_test[['age', 'fare']] = scaler.transform(X_test[['age', 'fare']])


# from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')
logit.fit(X_train, y_train)
y_pred = logit.predict(X_train)
y_pred_proba = logit.predict_proba(X_train)
X_train['prediction'] = logit.predict(X_train[['pclass','age','fare','sibsp','parch']])
# (y_train.survived == X_train.prediction).sum() / y_train.shape[0]
# logit.score(X_train[['pclass','age','fare','sibsp','parch']], y_train.survived)
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train.drop(columns='prediction'), y_train)))
print(confusion_matrix(y_train, y_pred))
df = pd.DataFrame(confusion_matrix(y_train.survived, X_train.prediction),
             columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])
print(classification_report(y_train, y_pred))
TN = df['Pred -'][0] # 190
FP = df['Pred +'][0] #103
FN = df['Pred -'][1] # 50
TP = df['Pred +'][1] # 156
total = TN + FP + FN + TP

print('True Negative = ', TN)
print('False Positive = ', FP)
print('False Negative = ', FN)
print('True Positive = ', TP)
print('Total = ', total)

# Accuracy = # correct / total 
#          = (true positive + true negative) / total
accuracy = (TP + TN) / total
print('Accuracy = ', accuracy)

# Recall = Sensitivity
#      = true positive rate 
#      = true positive / (true positive + false negative) 
recall = TP / (TP + FN)
print('Recall = ', recall)

# Specificity = false positive rate
#      = false positive / (false positive + true negative)
specificity = FP / (FP + TN)
print('Specificity = ', specificity)

# true negative rate = true negative / (true negative + false positive)
trueneg = TN / (TN + FP)
print('True Negative Rate = ', trueneg)

# false negative rate = false negative / (false negaitve + true positive)
falseneg = FN / (FN + TP)
print('False Negative Rate = ', falseneg)

# precision = true positive / (true positive + false positive)
precision = TP / (TP + FP)
print('Precision = ', precision)

f1 = (precision + recall) / 2
print('f1-score is ', f1)

died = TN + FP
lived = TP + FN
print(died, 'people died and', lived, 'people lived.')

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

Accuracy of Logistic Regression classifier on training set: 0.69
[[190 103]
 [ 50 156]]
              precision    recall  f1-score   support

           0       0.79      0.65      0.71       293
           1       0.60      0.76      0.67       206

   micro avg       0.69      0.69      0.69       499
   macro avg       0.70      0.70      0.69       499
weighted avg       0.71      0.69      0.70       499

True Negative =  190
False Positive =  103
False Negative =  50
True Positive =  156
Total =  499
Accuracy =  0.6933867735470942
Recall =  0.7572815533980582
Specificity =  0.3515358361774744
True Negative Rate =  0.6484641638225256
False Negative Rate =  0.24271844660194175
Precision =  0.6023166023166023
f1-score is  0.6797990778573303
293 people died and 206 people lived.


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
